In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [3]:
import sys
sys.path.insert(0,'/usr/local/bin/chromedriver')
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import json
import pprint
import time
import pickle
import re


Homepage j-pop manga e salvo nella variabile a il numero di manga pubblicati 

In [34]:
chrome_options=webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=chrome_options)
wd.set_window_size(1900, 1200)
wd.maximize_window()
wd.get("https://www.j-pop.it/293-j-pop-manga#/")

In [35]:
a = wd.find_elements_by_css_selector("#center_column > div > div.main-container.col2-left-layout > div > div.col-main > div.resumecat.category-product-count")

In [36]:
print(a[0].text) #numero prodotti

Ci sono 2726 prodotti.


In [110]:
#lista_link_manga= []
#len_manga_page= wd.find_elements_by_css_selector("#grid_view_product > li > div > div > div > div.name > a")
#for _ in range(len(len_manga_page)):
#    lista_link_manga.append(wd.find_element_by_css_selector(f"#grid_view_product > li:nth-child({_+1}) > div > div > div > div.name> a").get_attribute("href"))

In [40]:
num_pagine_sito = int(wd.find_element_by_css_selector("#pagination > ul > li:nth-child(6) > a").text)

Ottengo numero pagine sito web 

In [41]:
num_pagine_sito

303

Creo lista con pagine sito

In [56]:
lista_link_manga= []
for _ in range(num_pagine_sito):
  lista_link_manga.append(f"https://www.j-pop.it/293-j-pop-manga?selected_filters=page-{_+1}")

Ciclo per ogni pagina e salvo i sottolink per i manga

In [ ]:
manga_titoli_list=[]
for i in range(len(lista_link_manga)):
  time.sleep(10)
  print(lista_link_manga[i])
  chrome_options=webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  wd.set_window_size(1900, 1200)
  wd.maximize_window()
  wd.get(lista_link_manga[i])
  #elems = wd.find_elements_by_xpath("//*[@id=\"grid_view_product\"]/li/div/div/div/div[2]/a")
  elems = wd.find_elements_by_css_selector("#grid_view_product > li > div > div > div > div.name > a")
  try:
    for elem in elems:
      manga_titoli_list.append(elem.get_attribute("href"))
  except Exception as e :
    print(e)
    continue
  wd.close()

In [63]:
print(len(manga_titoli_list))

2719


Salvo le liste con le pagine e con i link per velocizzare in caso di blocco scaper

In [20]:
with open('lista_link_manga.txt', 'wb') as fp:
    pickle.dump(lista_link_manga, fp)

In [64]:
with open('manga_titoli_list.txt', 'wb') as fp:
    pickle.dump(manga_titoli_list, fp)

In [6]:
from google.colab import files
from google.colab import drive
drive.mount('drive')
!cp lista_link_manga.txt "drive/My Drive/"
!cp manga_titoli_list.txt "drive/My Drive/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive
cp: cannot stat 'lista_link_manga.txt': No such file or directory
cp: cannot stat 'manga_titoli_list.txt': No such file or directory


In [10]:
#with open ('drive/My Drive/manga_titoli_list.txt', 'rb') as fp:
#    lista_link_manga = pickle.load(fp)

In [9]:
#lista_link_manga

In [ ]:
#with open ('C:/Users/pucci/Documents/Programs/lista_link_manga.txt', 'rb') as fp:
    # lista_link_manga = pickle.load(fp)

In [22]:
import re
import requests

loop per ogni manga e genero il dizionario con le informazioni

In [ ]:
manga_dict = []
!mkdir -p "drive/My Drive/Images" 
for _ in range(len(manga_titoli_list)):
    try:
        time.sleep(3)
        chrome_options=webdriver.ChromeOptions()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        wd = webdriver.Chrome('chromedriver',options=chrome_options)
        wd.set_window_size(1900, 1200)
        wd.maximize_window()
        wd.get(manga_titoli_list[_])
        titolo =wd.find_element_by_class_name("product-name")
        print(titolo.text)
        prezzo =wd.find_elements_by_css_selector("#buy_block > div.price-box")
        img = wd.find_element_by_class_name("main-thumbnail")
        button =wd.find_elements_by_css_selector("#center_column > div > div.main-container.col1-layout > div > div > div.product-view.layout_ > div.product-collateral > ul#more_info_tabs > li > a")
        button[1].click()
        scheda_tecnica = wd.find_elements_by_css_selector("#center_column > div > div.main-container.col1-layout > div > div > div.product-view.layout_ > div.product-collateral > div > ul > li")
        temp_scheda_tecnica = []
        for i in range(len(scheda_tecnica)):
            temp_scheda_tecnica.append(scheda_tecnica[i].text.split("\n"))
        button[0].click()
        di_cosa_si_tratta = wd.find_elements_by_css_selector("#center_column > div > div > div > div > div> div.product-collateral")
        manga_dict.append({'url':manga_titoli_list[_],
              'titolo':titolo.text,
              'prezzo':prezzo[0].text.split("\n"),
              'descrizione':di_cosa_si_tratta[0].text.strip("\n"),
              'scheda_tecnica':temp_scheda_tecnica,
              'img_file':img.get_attribute("href")
              })
        img_file= requests.get(img.get_attribute("href"),stream=True)
        if img_file.status_code ==  200:
            #with open("drive/My Drive/Images" + str(titolo.text)+ ".jpg","wb") as f:
            with open("drive/My Drive/image/" + img.get_attribute("href").split("/")[-1]+ ".jpg","wb") as f:
                f.write(img_file.content)
    except Exception as e :
        print(e)
        continue

In [76]:
#lista_link_manga

In [77]:
#manga_dict

In [75]:
ds_manga_dict.shape

(2692, 6)

In [92]:
ds_manga_dict.head()

,url,titolo,prezzo,descrizione,scheda_tecnica,img_file
0,https://www.j-pop.it/barbara/5310-barbara-001....,Barbara 001,"[Prezzo: 12,50 €, Riferimento: 9788834900543]","Di cosa si tratta\nScheda tecnica\n“Barbara”, ...","[[Autore: Moto Hagio], [Numero volumi: Serie d...",https://www.j-pop.it/5204-thickbox_default/bar...
1,https://www.j-pop.it/tokyo-after-hours/5309-to...,Tokyo After Hours - BOX (1-3),"[Prezzo: 20,70 €, Riferimento: 9788834901649]",Di cosa si tratta\nScheda tecnica\nNel mondo s...,"[[Autore: Yutha Nishio], [Numero volumi: Cofan...",https://www.j-pop.it/5203-thickbox_default/tok...
2,https://www.j-pop.it/the-proper-way-to-write-l...,The Proper Way to Write Love,"[Prezzo: 6,90 €, Riferimento: 9788834903551]",Di cosa si tratta\nScheda tecnica\nOgeretsu Ta...,"[[Autore: Ogeretsu Tanaka], [Numero volumi: Vo...",https://www.j-pop.it/5202-thickbox_default/the...
3,https://www.j-pop.it/danmachi-sword-oratoria/5...,Danmachi Sword Oratoria 012,"[Prezzo: 5,90 €, Riferimento: 9788834901410]",Di cosa si tratta\nScheda tecnica\nLefiya perd...,"[[Scrittore: Fujino Omori], [Disegnatore: Taka...",https://www.j-pop.it/5201-thickbox_default/dan...
4,https://www.j-pop.it/so-i-m-a-spider-so-what/5...,"So I'm a Spider, So What? 001","[Prezzo: 6,50 €, Riferimento: 9788834903346]",Di cosa si tratta\nScheda tecnica\nHo sentito ...,"[[Autore: Okina Baba], [Scrittore: Okina Baba,...",https://www.j-pop.it/5196-thickbox_default/so-...


In [102]:
new_descrizione = []
for index,row in ds_manga_dict.iterrows():
  new_descrizione.append(row['descrizione'].split("\n")[-1])

In [105]:
ds_manga_dict2['descrizione'] =new_descrizione

In [104]:
ds_manga_dict2 = ds_manga_dict.copy()

In [106]:
ds_manga_dict2

,url,titolo,prezzo,descrizione,scheda_tecnica,img_file
0,https://www.j-pop.it/barbara/5310-barbara-001....,Barbara 001,"[Prezzo: 12,50 €, Riferimento: 9788834900543]","“Barbara”, un nome misterioso che sconvolge le...","[[Autore: Moto Hagio], [Numero volumi: Serie d...",https://www.j-pop.it/5204-thickbox_default/bar...
1,https://www.j-pop.it/tokyo-after-hours/5309-to...,Tokyo After Hours - BOX (1-3),"[Prezzo: 20,70 €, Riferimento: 9788834901649]",Rapite dalla musica e dalla personalità l'una ...,"[[Autore: Yutha Nishio], [Numero volumi: Cofan...",https://www.j-pop.it/5203-thickbox_default/tok...
2,https://www.j-pop.it/the-proper-way-to-write-l...,The Proper Way to Write Love,"[Prezzo: 6,90 €, Riferimento: 9788834903551]",Ogeretsu Tanaka continua a esplorare le mille ...,"[[Autore: Ogeretsu Tanaka], [Numero volumi: Vo...",https://www.j-pop.it/5202-thickbox_default/the...
3,https://www.j-pop.it/danmachi-sword-oratoria/5...,Danmachi Sword Oratoria 012,"[Prezzo: 5,90 €, Riferimento: 9788834901410]",Lefiya perde infine la pazienza per le continu...,"[[Scrittore: Fujino Omori], [Disegnatore: Taka...",https://www.j-pop.it/5201-thickbox_default/dan...
4,https://www.j-pop.it/so-i-m-a-spider-so-what/5...,"So I'm a Spider, So What? 001","[Prezzo: 6,50 €, Riferimento: 9788834903346]","Con la prima tiratura, in omaggio una variant ...","[[Autore: Okina Baba], [Scrittore: Okina Baba,...",https://www.j-pop.it/5196-thickbox_default/so-...
...,...,...,...,...,...,...
2687,https://www.j-pop.it/immortal-regis/197-immort...,Immortal Regis 01,"[Prezzo: 5,90 €]",Anche la morte non è poi così tranquilla! Lo s...,"[[Autore: Gaonbi], [Numero volumi: Serie di 06...",https://www.j-pop.it/196-thickbox_default/immo...
2688,https://www.j-pop.it/immortal-regis/198-immort...,Immortal Regis 02,"[Prezzo: 5,90 €]",Anche la morte non è poi così tranquilla! Lo s...,"[[Autore: Gaonbi], [Numero volumi: Serie di 06...",https://www.j-pop.it/197-thickbox_default/immo...
2689,https://www.j-pop.it/immortal-regis/199-immort...,Immortal Regis 03,"[Prezzo: 5,90 €]","Dopo la vittoria sul Manticore, una figura eva...","[[Autore: Gaonbi], [Numero volumi: Serie di 06...",https://www.j-pop.it/198-thickbox_default/immo...
2690,https://www.j-pop.it/dungeon-food/4248-dungeon...,Dungeon Food 002,"[Prezzo: 6,90 €, Riferimento: 9788832750638]",La sgangherata compagnia di avventurieri ha in...,"[[Autore: Ryoko Kui], [Pagine: 208], [Formato:...",https://www.j-pop.it/4102-thickbox_default/dun...


In [107]:
import pandas as pd
ds_manga_dict = pd.DataFrame(manga_dict)
ds_manga_dict2.to_csv("drive/My Drive/ds_manga_dict.csv",index=False,header=True,sep="\t")